In [1]:
"""0. General libraries"""
import pandas as pd
import numpy as np


In [2]:
"""1. Import dataset (from https://archive.ics.uci.edu/dataset/2/adult)"""
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
# url = "https://archive.ics.uci.edu/static/public/2/data.csv"
# adult = pd.read_csv(url) 
adult = fetch_ucirepo(id=2)

In [3]:
X = adult.data.features 
# flatten
y = adult.data.targets.values.ravel()

In [4]:
X

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States
48838,64,NaN,321403,HS-grad,9,Widowed,NaN,Other-relative,Black,Male,0,0,40,United-States
48839,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States
48840,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States


In [7]:
X.value_counts()

age  workclass         fnlwgt  education     education-num  marital-status      occupation         relationship   race   sex     capital-gain  capital-loss  hours-per-week  native-country
25   Private           308144  Bachelors     13             Never-married       Craft-repair       Not-in-family  White  Male    0             0             40              Mexico            3
21   Private           243368  Preschool     1              Never-married       Farming-fishing    Not-in-family  White  Male    0             0             50              Mexico            3
25   Private           195994  1st-4th       2              Never-married       Priv-house-serv    Not-in-family  White  Female  0             0             40              Guatemala         3
43   Private           195258  HS-grad       9              Married-civ-spouse  Craft-repair       Husband        White  Male    0             0             40              United-States     2
30   Private           180317  Assoc-voc

In [10]:
X = pd.get_dummies(X, 
    columns=['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country'])

# income > 50k will be set to 1 and <=50k will be set to 0
y = (y == '>50K.').astype(float)

In [5]:
y

array(['<=50K', '<=50K', '<=50K', ..., '<=50K.', '<=50K.', '>50K.'],
      dtype=object)

In [11]:
from sklearn.model_selection import train_test_split

# train/test split (75% training, 25% testing, seed=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)
print(np.bincount(y_train.astype(int)))


[33726  2905]


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV 
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# set of values to try
lambda_values = [0.001, 0.01, 0.1, 1.0]
C_values = [1 / l for l in lambda_values]

# make a pipeline to set input features to have mean=0 and std=1 (z-score normalize)
lr_pipeline = make_pipeline(
    StandardScaler(),
    LogisticRegression(
        penalty = 'l2',
        tol = 1e-4,
        random_state = 0,
        # Uses batch optimization and approximates second derivatives
        solver = 'lbfgs',
        max_iter = 1000,
        # Balance since too many samples are in <=50k category (less weight to <=50k and more weight to >50k)
        class_weight = 'balanced'
    )
)
# Paramter grid of C
C_dict = {'logisticregression__C': C_values}

# recipe for cross validation training
cross_val = GridSearchCV(
    estimator = lr_pipeline, #1
    param_grid = C_dict, #4
    # log loss (closer to 0 = better)
    scoring = 'neg_log_loss',
    # Use all CPUs. Change this value when using GPUs
    n_jobs = -1,
    # pick the best model 
    refit = True,
    # 4 folds to match the 75% training 25% testing structure
    cv = 4,
    # Only show progress bar
    verbose = 1,
    # throws and error when training fails
    error_score = 'raise',
    # returns how fit the model is
    return_train_score = True
)

In [13]:
# Cross validation training
cross_val.fit(X_train, y_train)

Fitting 4 folds for each of 4 candidates, totalling 16 fits


GridSearchCV(cv=4, error_score='raise',
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('logisticregression',
                                        LogisticRegression(class_weight='balanced',
                                                           max_iter=1000,
                                                           random_state=0))]),
             n_jobs=-1,
             param_grid={'logisticregression__C': [1000.0, 100.0, 10.0, 1.0]},
             return_train_score=True, scoring='neg_log_loss', verbose=1)

In [16]:
cross_val.best_estimator_.named_steps['logisticregression'].n_iter_

array([29])

In [17]:
import torch
a = torch.arange(9, dtype= torch.float) - 4
b = a.reshape((3, 3))
torch.norm(a)

tensor(7.7460)

In [18]:
a

tensor([-4., -3., -2., -1.,  0.,  1.,  2.,  3.,  4.])

In [19]:
c=a*a
c

tensor([16.,  9.,  4.,  1.,  0.,  1.,  4.,  9., 16.])

In [21]:
c.sum()**0.5

tensor(7.7460)

In [23]:
"""0. General libraries"""
import pandas as pd
import numpy as np


"""1. Import dataset (from https://archive.ics.uci.edu/dataset/2/adult)"""
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
# url = "https://archive.ics.uci.edu/static/public/2/data.csv"
# adult = pd.read_csv(url) 
adult = fetch_ucirepo(id=2)


"""2. Data Separation"""
# data (as pandas dataframes) 
X = adult.data.features 
# flatten
y = adult.data.targets.values.ravel()

## print(y)

# Converting all categorical columns to numbers (floats)
X = pd.get_dummies(X, 
    columns=['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country'])

# income > 50k will be set to 1 and <=50k will be set to 0
y = (y == '>50K.').astype(float)


## print(X)
## print(y)

# metadata 
## print(adult.metadata) 
  
# variable information 
## print(adult.variables) 

# for exp()
## X = np.asarray(X)

## print(X)

## raise Exception("Stopping here: condition not met")

"""3. Data Splitting"""
from sklearn.model_selection import train_test_split

# train/test split (75% training, 25% testing, seed=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)


# checking the balance of training labels
# [33726, 2905]
print(np.bincount(y_train.astype(int)))


"""4. Cross Validation"""
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV 
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# set of values to try
lambda_values = [0.001, 0.01, 0.1, 1.0]
C_values = [1 / l for l in lambda_values]

# make a pipeline to set input features to have mean=0 and std=1 (z-score normalize)
lr_pipeline = make_pipeline(
    StandardScaler(),
    LogisticRegression(
        penalty = 'l2',
        tol = 1e-4,
        random_state = 0,
        # Uses batch optimization and approximates second derivatives
        solver = 'lbfgs',
        max_iter = 1000,
        # Balance since too many samples are in <=50k category (less weight to <=50k and more weight to >50k)
        class_weight = 'balanced'
    )
)

# Paramter grid of C
C_dict = {'logisticregression__C': C_values}

# recipe for cross validation training
cross_val = GridSearchCV(
    estimator = lr_pipeline,
    param_grid = C_dict,
    # log loss (closer to 0 = better)
    scoring = 'neg_log_loss',
    # Use all CPUs. Change this value when using GPUs
    n_jobs = -1,
    # pick the best model 
    refit = True,
    # 4 folds to match the 75% training 25% testing structure
    cv = 4,
    # Only show progress bar
    verbose = 1,
    # throws and error when training fails
    error_score = 'raise',
    # returns how fit the model is
    return_train_score = True
)

# Cross validation training
cross_val.fit(X_train, y_train)

# Best C value:  1.0
# Best average validation score:  -0.5233608466485005
print("Best C value: ", cross_val.best_params_['logisticregression__C'])
print("Best average validation score: ", cross_val.best_score_)

# Set C and lambda to 1
best_c = 1.0
best_lam = 1 / best_c

"""5. Optimal model"""
# Use batch gradient descent; z-score normalize
optimal_pipeline = make_pipeline(
    StandardScaler(),
    LogisticRegression(
        penalty = 'l2',
        # set it to as close to 0 as possible (for testing purposes, set it to 1e-5)
        # during official run, set it to 1e-9 
        # 1e-12: takes about 2 hours and goes through 190619 epochs
        tol = 1e-2, #temp: change back to 1e-5
        C = best_c,
        # sets the seed for random number generator to get same results for every run
        random_state = 0,
        # try saga or lbfgs
        solver = 'saga',
        max_iter = 1000000,
        # show progress messages (hide when solver=saga)
        verbose = 1,
        class_weight = 'balanced'
    )
)

# train optimal model
optimal_pipeline.fit(X_train, y_train)

# save optimal model
optimal_model = []
optimal_model.append(optimal_pipeline.named_steps['logisticregression'])

# attributes of the trained optimal model
opt_class = optimal_pipeline.classes_
opt_iter = optimal_pipeline.named_steps['logisticregression'].n_iter_
opt_pred = optimal_pipeline.predict(X_test)
opt_prob = optimal_pipeline.predict_proba(X_test)

print("Class index to label: ", opt_class)
print("Number of iterations: ", opt_iter)
print("Predicted class of testing input dataset: ", opt_pred)
print("Approximate probabilities for each class: ", opt_prob)


"""6. 1000 random epsilon-vicinal models"""
# parallel programming 
from joblib import Parallel, delayed
# sigmoid
from scipy.special import expit

# set epsilon to 0.01 (perfect value according to the paper)
epsilon = 0.01

# gradient after training
# regularized logistic loss function:
### L(w) = -(1/n)*sum[i=1 n] (yi*log(ypi)) + (1-yi)*log(1-ypi)) + lambda/2*||w||^2
# gradient:
### L'(w) = (1/n)*(X^T)*(yp-y) + lambda*w
# ypi = 1/(1+e^(-(w^T)*xi))
# yp = 1/(1+e^-Xw)
def gradient(X, y, w, l, b=None):
    X = np.asarray(X, dtype=np.float64)
    w = np.asarray(w, dtype=np.float64)

    if b is not None:
        # add bias column
        X = np.hstack([X, np.ones((X.shape[0], 1))])

        # append intercept to weight vector
        w = np.append(w, b)

    n = X.shape[0]

    z = X @ w
    z = np.asarray(z, dtype=np.float64)

    ## yp = 1 / (1 + np.exp(-z))
    yp = expit(z)

    reg = l * w #np.linalg.norm(w, ord=2) l_2 torch.norm
       
    if b is not None:
        reg[-1] = 0

    G = (1 / n) * (X.T @ (yp-y)) + reg
    return G

# function for running one model

# Use batch gradient descent; z-score normalize
vicinal_pipeline = make_pipeline(
    StandardScaler(),
    LogisticRegression(
        penalty = 'l2',
        # Make sure all 1000 epochs are run
        tol = 1e-12,
        C = best_c,
        # sets the seed for random number generator to get same results for every run
        random_state = 42,
        solver = 'saga',
        max_iter = 1000,
        # don't show progress messages
        verbose = 1,
        class_weight = 'balanced'
    )
)

vicinal_pipeline.fit(X_train, y_train)

print("LBFGS iterations:", vicinal_pipeline.named_steps['logisticregression'].n_iter_)

# Turn model weights and biases into 1d array
w_vicinal = vicinal_pipeline.named_steps['logisticregression'].coef_.flatten()
b_vicinal = vicinal_pipeline.named_steps['logisticregression'].intercept_[0]

# Scaling X_train
X_scale = vicinal_pipeline.named_steps['standardscaler'].transform(X_train)

gradient_vicinal = gradient(X_scale, y_train, w_vicinal, best_lam, b_vicinal)

# L2 norm of the gradient
gradient_vicinal_mag = np.linalg.norm(gradient_vicinal)

print("gradient of model: ", gradient_vicinal_mag)






[33726  2905]
Fitting 4 folds for each of 4 candidates, totalling 16 fits
Best C value:  1.0
Best average validation score:  -0.5233608466484991
convergence after 41 epochs took 2 seconds
Class index to label:  [0. 1.]
Number of iterations:  [41]
Predicted class of testing input dataset:  [1. 0. 0. ... 0. 0. 0.]
Approximate probabilities for each class:  [[0.42346023 0.57653977]
 [0.61198776 0.38801224]
 [0.82085467 0.17914533]
 ...
 [0.85031475 0.14968525]
 [0.68662231 0.31337769]
 [0.97009564 0.02990436]]
max_iter reached after 48 seconds
LBFGS iterations: [1000]
gradient of model:  2.2832694651159953


C:\Users\hskay\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [24]:
gradient_vicinal

array([ 0.41275547,  0.02387797,  0.34430144,  0.29147713,  0.1658326 ,
        0.34690204, -0.78088467,  0.01400033, -0.04852484, -0.03952046,
       -0.14735828, -0.02444522, -0.13007847, -0.08609742,  0.00808175,
       -0.1380737 , -0.20822106, -0.07482838, -0.0888063 , -0.04798201,
       -0.12457694, -0.09727447,  0.02203034,  0.05819644,  0.15504321,
        0.04038534, -0.04213846,  0.15466383,  0.00279197,  0.05080805,
        0.02743055, -0.26282623,  0.01251656,  0.87015225, -0.07876344,
       -0.60914531, -0.14234401, -0.14137018, -0.7815676 , -0.09339839,
        0.02127166, -0.02256064,  0.13870734, -0.16367209, -0.15345684,
       -0.09462344, -0.36063176, -0.07262788,  0.12631786,  0.02522252,
        0.00492871,  0.05810902, -0.01664289,  0.00990007,  0.20920463,
       -0.15297203, -0.22196113, -0.03953373,  0.10595863, -0.02503695,
        0.06579846, -0.10798805,  0.05879458,  0.04939709, -0.15582314,
        0.15582314, -0.95514128, -0.03684081, -0.0442346 , -0.06

In [25]:
gradient_vicinal_mag

2.2832694651159953

In [ ]:
# check your preprocess  
    # pipeline
# (pytorch) for the ml algorithm we should use which one the most customazible # future
# check equations and continue to work on logistic regression


In [ ]:
LogisticRegression(solve='lgfbs')